In [1]:
import os
import re
from collections import deque
import numpy as np
import wave, struct
from tqdm import tqdm

In [2]:
fileName = 'a'

In [3]:
fileInBin = open("out_bin_p.log", "rt") 

lineBin = fileInBin.readline()

In [4]:
def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return np.concatenate((even_fields, odd_fields), axis=0)

bufL0 = deque()
bufR0 = deque()
bufL1 = deque()
bufR1 = deque()
bufL2 = deque()
emptyLine = np.zeros(14, dtype=bool)

#kluge init queues
for i in range((16*5)-1):
    bufL0.append(emptyLine)

for i in range((16*4)-1):
    bufR0.append(emptyLine)

for i in range((16*3)-1):
    bufL1.append(emptyLine)

for i in range((16*2)-1):
    bufR1.append(emptyLine)
    
for i in range(16-1):
    bufL2.append(emptyLine)

def bitsToInt(bits):
    a = 0
    for i in range(0, len(bits)):
        a = bits[i] | a << 1
    return a
    
def deinterlive(block):
    block = block[4:88]
    
    bufL0.append(block[:14])
    bufR0.append(block[14:28])
    bufL1.append(block[28:42])
    bufR1.append(block[42:56])
    bufL2.append(block[56:70])
    
    return bufL0.popleft(), bufR0.popleft(), bufL1.popleft(), bufR1.popleft(), bufL2.popleft(), block[70:84] #L0, R0, L1, R1, L2, R2

In [5]:
bar = tqdm(total=486*625)
bar.set_description("decoding")

file = wave.open(fileName + '.wav', mode='wb')
file.setnchannels(2) 
file.setsampwidth(2)
file.setframerate(44100.0)

file.setframerate(44100)

decoding:   0%|                                                                                  | 0/303750 [00:00<?, ?it/s]

In [6]:
def convertDataBlock(data):
  data = list(data.replace('-', '1').replace('#', '0'))
  data = [bool(int(numeric_string)) for numeric_string in data]
  return data

while lineBin:
    bar.update(1)
    typeLine=re.match('\"?\[(V2D)\]', lineBin)
    if typeLine != None:
        typeLine = typeLine.group(1)
        if typeLine == 'V2D':
            parse = re.match('.?\[V2D\].*F\[(\d+)\].*L\[(\d+)\].*\[([#-]{137})\].*(CRC OK|No PCM!|BD CRC!)', lineBin)
            if parse and parse.group(3):
                #compare frame with log output
                                
                if not parse.group(4) == 'No PCM!':
                    L0, R0, L1, R1, L2, R2 = deinterlive(convertDataBlock(parse.group(3)))
                    file.writeframes(struct.pack('<H', (bitsToInt(L0) << 2)))
                    file.writeframes(struct.pack('<H', (bitsToInt(R0) << 2)))
                    file.writeframes(struct.pack('<H', (bitsToInt(L1) << 2)))
                    file.writeframes(struct.pack('<H', (bitsToInt(R1) << 2)))
                    file.writeframes(struct.pack('<H', (bitsToInt(L2) << 2)))
                    file.writeframes(struct.pack('<H', (bitsToInt(R2) << 2)))
                    
    lineBin = fileInBin.readline()

bar.close()


decoding: : 306884it [01:24, 3632.09it/s]                                                                                   
